Cosmetecor MultiElectrode EMF Dataset
==================

Multi-electrode ground measurement technique was introduced by Docent D. Kuznetsov in the 1980s as an earthquake prediction method. There are currently several stations collecting EMF data using the Kuznetsov's technique. These data are archived in an FTP site: 
<ftp://cosmetecor.org/archive_all/>

This Notebook contains the production code for
* Automatically extracting .LOG files from the FTP site
* Gluing these .LOG files into yearly .csv files
* Creating a HDF5 file with the years as the groups, and timestamp and EMF as datasets under these groups

In [1]:
# As usual, a bit of setup

import time, os, json
import numpy as np
import matplotlib.pyplot as plt
import csv
import h5py
import sys

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 6.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

## FTP File Extraction

The data collected at each station is archived in folders "/archive_all/YEAR/STATION_NAME/" at the FTP site. The files are a mix of .LOG and .7Z files (.7Z files can be decompressed into .LOG files using Winzip).

For each station, the download should generate XXXX .LOG files (each representing 1 day of data). Each .LOG file has 86400 rows of data (24hr x 60min x 60sec). Each row consists of:
(1) Datetime
(2) 32 measurements

In [ ]:
import ftplib

#Open ftp connection
ftp = ftplib.FTP('ftp.cosmetecor.org', 'anonymous','luke8liem@gmail.com')


# ftp_dirs = ["/archive_all/2013/S1-IMFSET/",
#             "/archive_all/2014/S1-IMFSET/",
#             "/archive_all/2015/S1-IMFSET_2015/",
#             "/archive_all/2016/S1-IMFSET_2016/",
#             "/archive_all/2017/S1-IMFSET_2017/"]
# local_dir = "cosmetecor data/s1-imfset/"

#ftp_dirs = ["/archive_all/2012/s1-DESP-PK/",
#            "/archive_all/2013/s1-DESP-PK/",
#            "/archive_all/2014/s1-DESP-PK/",
#            "/archive_all/2015/s1-DESP-PK_2015/",
#            "/archive_all/2016/s1-DESP-PK_2016/",
#            "/archive_all/2017/s1-DESP-PK_2017/"]
#local_dir = "cosmetecor data/s1-DESP-PK/"

#ftp_dirs = ["/archive_all/2012/s1-DESP-UZ/",
#            "/archive_all/2013/s1-DESP-UZ/",
#            "/archive_all/2014/s1-DESP-UZ/",
#            "/archive_all/2015/s1-DESP-UZ_2015/",
#            "/archive_all/2016/s1-DESP-UZ_2016/",
#            "/archive_all/2017/s1-DESP-UZ_2017/"]
#local_dir = "cosmetecor data/s1-DESP-UZ/"

# ftp_dirs = ["/archive_all/2012/s2-imfset/",
#            "/archive_all/2013/s2-imfset/",
#            "/archive_all/2014/s2-imfset/",
#            "/archive_all/2015/s2-imfset_2015/",
#           "/archive_all/2016/s2-imfset_2016/",
#           "/archive_all/2017/s2-imfset_2017/"]
# local_dir = "cosmetecor data/s2-imfset/"

#ftp_dirs = ["/archive_all/2012/s3-imfset/",
#            "/archive_all/2013/s3-imfset/",
#            "/archive_all/2014/s3-imfset/",
#            "/archive_all/2015/s3-imfset_2015/",
#            "/archive_all/2016/s3-imfset_2016/",
#            "/archive_all/2017/s3-imfset_2017/"]
#local_dir = "cosmetecor data/s3-imfset/"

#ftp_dirs = ["/archive_all/2013/s5-altai/",
#            "/archive_all/2014/s5-altai/",
#            "/archive_all/2015/s5-altai_2015/",
#            "/archive_all/2016/s5-altai_2016/",
#            "/archive_all/2017/s5-altai_2017/"]
#local_dir = "D:/geocosmo/cosmetecor data/s5-altai/"

#ftp_dirs = ["/archive_all/2013/s7-imfset/",
#            "/archive_all/2014/s7-imfset/",
#            "/archive_all/2015/s7-imfset_2015/",
#            "/archive_all/2016/s7-imfset_2016/",
#            "/archive_all/2017/s7-imfset_2017/"]
#local_dir = "cosmetecor data/s7-imfset/"


ftp_dirs = ["/archive_all/2012/s4-esso/",
            "/archive_all/2013/s4-esso/",
            "/archive_all/2014/s4-esso/",
            "/archive_all/2015/s4-esso_2015/",
            "/archive_all/2016/s4-esso_2016/"]
local_dir = "D:/geocosmo/cosmetecor data/s4-esso/"

for ftp_dir in ftp_dirs:
    ftp.cwd(ftp_dir)

    # Generate a list of files names in the current directory
    files = []

    try:
        files = ftp.nlst()
    except ftplib.error_perm, resp:
        if str(resp) == "550 No files found":
            print "No files in this directory"
        else:
            raise

    # Extract all the files in the current directory into a local folder        
    for f in files:
        command = "RETR "+ f   # RETR Filename
        destination = local_dir+f
        print "Extracting %s" % f
        dFile = open(destination, "wb")
        ftp.retrbinary(command, dFile.write)
        dFile.close()               
   
ftp.quit()

Extracting S4-ESSO-121006.log
Extracting S4-ESSO-121007.log
Extracting S4-ESSO-121008.log
Extracting S4-ESSO-121009.log
Extracting S4-ESSO-121010.log
Extracting S4-ESSO-121011.log
Extracting S4-ESSO-121012.log
Extracting S4-ESSO-121013.log
Extracting S4-ESSO-121014.log
Extracting S4-ESSO-121015.log
Extracting S4-ESSO-121016.log
Extracting S4-ESSO-121017.log
Extracting S4-ESSO-121018.log
Extracting S4-ESSO-121019.log
Extracting S4-ESSO-121020.log
Extracting S4-ESSO-121021.log
Extracting S4-ESSO-121022.log
Extracting S4-ESSO-121023.log
Extracting S4-ESSO-121024.log
Extracting S4-ESSO-121025.log
Extracting S4-ESSO-121026.log
Extracting S4-ESSO-121027.log
Extracting S4-ESSO-121028.log
Extracting S4-ESSO-121029.log
Extracting S4-ESSO-121030.log
Extracting S4-ESSO-121031.log
Extracting S4-ESSO-121101.log
Extracting S4-ESSO-121102.log
Extracting S4-ESSO-121103.log
Extracting S4-ESSO-121104.log
Extracting S4-ESSO-121105.log
Extracting S4-ESSO-121106.log
Extracting S4-ESSO-121107.log
Extracting

## Consolidate Daily .LOG files --> Annual .csv File

Using the code below, daily .LOG files can be consolidated into yearly .csv files. As an example, for the Station 6 Chieti dataset, the following will be generated:

* s6-chieti2017.csv
* s6-chieti2016.csv
* s6-chieti2015.csv
* s6-chieti2014.csv
* s6-chieti2013.csv

In [3]:
import h5py
from data_util.data_analyze import *
import glob
import csv

# s1-imfset parameters
# years = ["13","14","15","16","17"]
# csv_filename_header = 's1-imfset20'
# log_filename_header = 'cosmetecor data\s1-imfset\S1-ELISOVO-'

# s2-imfset parameters
# years = ["12","13","14","15","16","17"]
# csv_filename_header = 's2-imfset20'
# log_filename_header = 'cosmetecor data\s2-imfset\IFPET-'

# s6-chieti parameters
# years = ["12","13","14","15","16","17"]
# csv_filename_header = 's6-chieti20'
# log_filename_header = 'cosmetecor data\s6-chieti\S6-CHIETI-'

# s3-imfset parameters
#years = ["12","13","14","15","16","17"]
#csv_filename_header = 's3-imfset20'
#log_filename_header = 'cosmetecor data\s3-imfset\S3-OKEAN-'

# s1-desp-pk parameters
years = ["12","13","14","15","16","17"]
csv_filename_header = 's1-desp-pk20'
log_filename_header = 'cosmetecor data\s1-DESP-PK\DESP-'

for year in years:
    csv_filename = csv_filename_header+year+'.csv'
    print csv_filename

    fout = open(csv_filename,'a')

    i = 0
    extraction_criteria = log_filename_header+year+'*.LOG'
    for filename in glob.glob(extraction_criteria):
        print filename
        for line in open(filename,'r'):
            fout.write(line)   
             
    fout.close()

s1-desp-pk2012.csv
cosmetecor data\s1-DESP-PK\DESP-120805.log
cosmetecor data\s1-DESP-PK\DESP-120806.log
cosmetecor data\s1-DESP-PK\DESP-120807.log
cosmetecor data\s1-DESP-PK\DESP-120808.log
cosmetecor data\s1-DESP-PK\DESP-120809.log
cosmetecor data\s1-DESP-PK\DESP-120810.log
cosmetecor data\s1-DESP-PK\DESP-120811.log
cosmetecor data\s1-DESP-PK\DESP-120812.log
cosmetecor data\s1-DESP-PK\DESP-120813.log
cosmetecor data\s1-DESP-PK\DESP-120814.log
cosmetecor data\s1-DESP-PK\DESP-120815.log
cosmetecor data\s1-DESP-PK\DESP-120816.log
cosmetecor data\s1-DESP-PK\DESP-120817.log
cosmetecor data\s1-DESP-PK\DESP-120818.log
cosmetecor data\s1-DESP-PK\DESP-120819.log
cosmetecor data\s1-DESP-PK\DESP-120820.log
cosmetecor data\s1-DESP-PK\DESP-120821.log
cosmetecor data\s1-DESP-PK\DESP-120822.log
cosmetecor data\s1-DESP-PK\DESP-120823.log
cosmetecor data\s1-DESP-PK\DESP-120824.log
cosmetecor data\s1-DESP-PK\DESP-120825.log
cosmetecor data\s1-DESP-PK\DESP-120826.log
cosmetecor data\s1-DESP-PK\DESP-120

## Create the HDF5 File

HDF5 file format allows us to create groups and datasets. Groups are similar to folders and datasets are essentially numpy arrays.

The code below takes the datetime and the EMF data in the .csv files (consolidated by year) and write them into datasets (timestamp and emf_data) organized by groups (year) in the .h5 file.

Take the Station 6 Chieti dataset as an example, the code takes the following yearly .csv files and write them into a single .h5 file 's6-chieti_emf.h5':

* s6-chieti2017.csv
* s6-chieti2016.csv
* s6-chieti2015.csv
* s6-chieti2014.csv
* s6-chieti2013.csv

Inside the .h5 file, there will be 5 groups each corresponding to a year of data:

* 2017
* 2016
* 2015
* 2014
* 2013

Within each group will be 2 datasets:

* timestamp - an 1-dim array of UNIX timestamp corresponding to the datetime of the EMF data (dtype = int)
* emf_data - a 28/32-dim array of EMF data (dtype = float32)

The code also do some basic data cleaning:
* If the number of columns are not as expected (most .LOG files contain 32 columns of EMF data, but some contains only 28), the code will print out the row of data with the mismatched column numbers and exit
* If the number of columns are matched, but there are missing data (usually empty string ''), the code will convert the empty string to NaN

In [9]:
import h5py
import sys
from data_util.data_analyze import *
from datetime import datetime
import calendar

#years = [2013,2014,2015,2016,2017]
#h5_filename = 'D:/geocosmo/cosmetecor data/s1-imfset_emf.h5'
#csv_filename_header = 'D:\geocosmo\cosmetecor data\s1-imfset'
# NUM_COLUMN = 32

years = [2012,2013,2014,2015,2016,2017]
h5_filename = 'D:/geocosmo/cosmetecor data/s1-desp-pk_emf.h5'
csv_filename_header = 'D:/geocosmo/cosmetecor data/s1-desp-pk'
NUM_COLUMN = 32

#years = [2012,2013,2014,2015,2016,2017]
#h5_filename = 'D:/geocosmo/cosmetecor data/s2-imfset_emf.h5'
#csv_filename_header = 'D:/geocosmo/cosmetecor data/s2-imfset'
#NUM_COLUMN = 28

#years = [2012,2013,2014,2015,2016,2017]
#h5_filename = 'D:/geocosmo/cosmetecor data/s3-imfset_emf.h5'
#csv_filename_header = 'D:\geocosmo\cosmetecor data\s3-imfset'
#NUM_COLUMN = 32

fout = h5py.File (h5_filename, 'w')

for year in years:
    filename = csv_filename_header+str(year)+'.csv'
    print filename
    
    # Count the number of rows of data
    print "Number of rows of data in %s:" % filename
    start = time.time()
    num = num_datapoints(filename,dict=False)
    end = time.time()
    print num
    print "Seconds taken is %.2f" % (end-start)

    # Create the arrays in memory to hold them
    # X - to hold the EMF measurements
    X = np.zeros((NUM_COLUMN,num), dtype=np.float32)
    print X.shape
    num_bytes = sys.getsizeof(X)
    print "Memory used for numpy array = %d bytes" % num_bytes

    # Timestamp - to hold the UNIX timestamp of the EMF data
    TimeStamp = np.zeros((1,num), dtype= 'int')
    print TimeStamp.shape
    num_bytes = sys.getsizeof(TimeStamp)
    print "Memory used for numpy array = %d bytes" % num_bytes

    # Transfer data from .csv file to memory
    start = time.time()
    with open(filename, 'rb') as f:
        reader = csv.reader(f, delimiter=' ')
        idx=0
        for row in reader:
            dt = datetime.strptime(row[0]+' '+row[1], "%Y.%m.%d %H:%M:%S")
            TimeStamp[0,idx] = datatime_2_timestamp(dt,utc=True)
            
            # This try-except statement identifies LOG files which do not contain the expected number of columns
            # of EMF data.
            try:
                for i, x in enumerate(row):
                    if x == '':
                        row[i] = np.NaN    # Replace any empty string '' with NaN
                X[:,idx] = row[2:NUM_COLUMN+2]
            except ValueError:
                print row
                fout.close()
                raise
            if idx < 2:     # Print out the first 2 rows
                print row
                print X
                print TimeStamp
            idx += 1
    f.close()
    end = time.time()
    print "Time to access the csv file to transfer %d column of data: %.2f" % (NUM_COLUMN,end-start)

    # Transfer data from memory to the datasets in .h5 file
    print "Time to write to the HDF5 file:"
    start = time.time()
    dset_name = str(year)+'/emf_data'
    tset_name = str(year)+'/timestamp'
    emf_data = fout.create_dataset(dset_name,(NUM_COLUMN,num), dtype="float32")
    emf_data[...] = X
    timestamp = fout.create_dataset(tset_name,(1,num), dtype="int")
    timestamp[...] = TimeStamp
    end = time.time()
    print end-start

fout.close()

# Release memory
TimeStamp = np.zeros((1,10), dtype= 'int')
X = np.zeros((NUM_COLUMN,10), dtype=np.float32)

D:/geocosmo/cosmetecor data/s1-desp-pk2012.csv
Number of rows of data in D:/geocosmo/cosmetecor data/s1-desp-pk2012.csv:
12027852
Seconds taken is 27.90
(32L, 12027852L)
Memory used for numpy array = 1539565168 bytes
(1L, 12027852L)
Memory used for numpy array = 48111520 bytes
['2012.08.05', '16:02:34', '797.269', '-715.163', '82.076', '-35.219', '46.418', '-19.607', '25.768', '33.194', '57.488', '185.958', '244.347', '752.491', '-44.95', '-43.251', '70.055', '14.843', '63.618', '47.103', '26.874', '7.029', '27.393', '42.046', '56.531', '28.246', '38.909', '175.951', '138.299', '138.39', nan, nan, nan, nan]
[[ 797.26898193    0.            0.         ...,    0.            0.            0.        ]
 [-715.1630249     0.            0.         ...,    0.            0.            0.        ]
 [  82.0759964     0.            0.         ...,    0.            0.            0.        ]
 ..., 
 [          nan    0.            0.         ...,    0.            0.            0.        ]
 [        

## Print Some Data

In [5]:
def printname(name):
    print name

file = h5py.File ('D:/geocosmo/cosmetecor data/s1-desp-pk_emf.h5', 'r')

# Explore group and datasets in the hdf5 file
file.visit(printname)
print file['2017/timestamp'].shape
print file['2017/emf_data'].shape

# print some data in dataset
dset = file['2017/emf_data'][:,10:15]
print dset

tset = file['2017/timestamp'][:,10:15]
print tset

file.close()

2012
2012/emf_data
2012/timestamp
2013
2013/emf_data
2013/timestamp
2014
2014/emf_data
2014/timestamp
2015
2015/emf_data
2015/timestamp
2016
2016/emf_data
2016/timestamp
2017
2017/emf_data
2017/timestamp
(1, 12066580)
(32, 12066580)
[[  -8.74499989   -8.74400043   -8.74300003   -8.74199963   -8.74199963]
 [-205.10499573 -205.10499573 -205.10499573 -205.10400391 -205.10400391]
 [-213.44799805 -213.44700623 -213.44700623 -213.44599915 -213.44500732]
 [  69.77300262   69.77300262   69.77300262   69.77300262   69.77200317]
 [-143.02099609 -143.02000427 -143.02000427 -143.01899719 -143.01899719]
 [  46.2140007    46.2140007    46.2140007    46.21300125   46.21300125]
 [ -93.57099915  -93.56999969  -93.56999969  -93.56900024  -93.56800079]
 [-133.09700012 -133.09700012 -133.09599304 -133.09599304 -133.09599304]
 [-224.11300659 -224.11199951 -224.11099243 -224.11000061 -224.10899353]
 [  -0.98900002   -0.98900002   -0.98900002   -0.99000001   -0.99000001]
 [-224.35600281 -224.35600281 -224.35

## Check Data Integrity

This makes sure that the EMF and timing data are not corrupted when converting from .csv files to .h5 groups and datasets.

THe code select 3 random row in the .csv file. Print out their content. It then goes to the .h5 file and get the corresponding 3 rows from the group/dataset.

It then performs a matrix subtraction of the 3 rows in the .h5 file from the 3 rows in the .csv, and print out the difference. It should be very small.

In [13]:
import random
import csv
from data_util.data_analyze import *
from datetime import datetime

filename = 'D:\geocosmo\cosmetecor data\s1-imfset2013.csv'
h5_filename = 'D:\geocosmo\cosmetecor data\s1-imfset_emf.h5'
h5_timestamp = '2013/timestamp'
h5_emf = '2013/emf_data'
NUM_COLUMNS = 32   # Make sure the number of columns of EMF data is correct!!!

# Count number of rows in .csv file
print "Number of rows of data in %s:" % filename
num = num_datapoints(filename,dict=False)
print num

# Randomly select 3 rows in a .csv file (a year of EMF data)
random_rows = []
random_rows =sorted(random.sample(xrange(num),3))

Timestamps = []
EMFs = np.zeros((NUM_COLUMNS,3))

f = open(filename, 'r')
reader = csv.reader(f, delimiter=' ')

print "3 random rows chosen from .csv file:"
j=0
for i, row in enumerate(reader):
    if i in random_rows:
        print "%s\n" %(row[0]+' '+row[1])
        dt = datetime.strptime(row[0]+' '+row[1], "%Y.%m.%d %H:%M:%S")
        Timestamps.append(datatime_2_timestamp(dt,utc=True))
        EMFs[:,j] =row[2:NUM_COLUMNS+2]
        j += 1

f.close()

print Timestamps
print EMFs

def index(array, item):
    for idx, val in np.ndenumerate(array):
        if val == item:
            return idx
 
# Access the same 3 rows in the .h5 file

file = h5py.File (h5_filename, 'r')
h5_EMFs = np.zeros((NUM_COLUMNS,3))

print file[h5_timestamp].shape
print file[h5_timestamp].dtype

for i, time in enumerate(Timestamps):
    idx = index(file[h5_timestamp],time)
    h5_EMFs[:,i] = file[h5_emf][:,idx[1]]

file.close()

print "The same 3 rows chosen in the .h5 file:"
print h5_EMFs

diff = np.sum(h5_EMFs - EMFs)
print ("The difference between the random data: ", diff)

Number of rows of data in D:\geocosmo\cosmetecor data\s1-imfset2013.csv:
25266966
3 random rows chosen from .csv file:
2013.10.11 09:24:29

2013.08.05 09:56:26

2013.11.04 02:02:47

[1381483469, 1375696586, 1383530567]
[[ -42.484  -48.146  -41.683]
 [ 113.684  122.018  117.304]
 [  30.171   37.571   27.283]
 [ -24.819  -26.649  -23.243]
 [   6.047    8.992    7.484]
 [   5.855    3.914    3.752]
 [  -6.181   -7.012   -4.657]
 [ -42.61   -41.829  -40.501]
 [ 112.932  132.191  117.397]
 [  38.891   34.294   40.314]
 [  13.929   56.202   15.88 ]
 [  46.323   24.468   51.334]
 [ -31.592  -37.083  -36.401]
 [  78.443   76.406   91.723]
 [  73.31    71.428   74.533]
 [  -6.928  -15.612  -10.801]
 [   1.561    1.551    1.834]
 [   0.76     0.604    0.54 ]
 [   1.81     1.323    1.321]
 [   1.026    0.805    0.748]
 [   1.529    1.236    1.148]
 [   1.96     1.494    1.496]
 [   0.816    0.673    0.684]
 [  10.31     8.099    7.468]
 [   0.779    0.756    0.6  ]
 [   1.831    2.086    1.24 ]
 

## Some Useful Code

The cells below are handy in case files are not closed, or resources are not released.

In [4]:
fout.close()

In [7]:
TimeStamp = np.zeros((1,10), dtype= 'int')
X = np.zeros((NUM_COLUMN,10), dtype=np.float32)

In [3]:
ftp.quit()

'221 Goodbye.'